# Basic code
This section introduces point cloud format and functions used frequently.

### TL;DR
- This tutorial uses the point cloud format of a two-dimensional array (shape: (number of points, channel size)).
- This tutorial frequently uses `tutlibs.io.Points` (reading and writing a file) and `tutlibs.visualization.JupyterVisualizer` (visualization for 3D representation).


## Format
This subsection introduces point cloud format. In this subsection, we use this package:

In [1]:
import numpy as np


Point cloud processing treats a point cloud as an array. As the following example, an array shows a point cloud with four points on three dimension space:

In [2]:
coords = np.array(
    [[1, 0, 0], [0, 0.5, 1], [1, 1, 0.1], [-1, 0.5, -0.3]], dtype=np.float32
)

print("array shape:{}".format(coords.shape))


array shape:(4, 3)


The first dimension of this array (`coords`) has points, and the second dimension has coordinate values of each point. Therefore, `4` of the array shape show number of points, and `3` show number of coordinate values on three dimension space.

A point cloud may contain additional information such as color information. If a color is assigned to each point in the above code, the color information is expressed as follows.

In [3]:
colors = np.array(
    [[255, 10, 20], [0, 89, 111], [74, 116, 121], [190, 95, 3]], dtype=np.int32
)

print("array shape:{}".format(colors.shape))


array shape:(4, 3)


The first dimension of this array (`colors`) has points, and the second dimension has color values of each point. Therefore, `4` of the array shape show number of points, and `3` show number of color values on three dimension space. Also, the index values of the first dimension correspond between the color and coordinate arrays. 

As you can see from the array of color and coordinate information, the point cloud array is represented in (number of points, channel size). The tutorial demonstrate various point cloud processing using this simple point cloud format. 

In this tutorial, the specifications of the point cloud array are as follows.
- This array shape is (number of points, channel size).
- Point cloud data is divided by type such as coordinates, colors.
- Arrays of the same point cloud correspond between the indices of the first dimension. (ex: i-th point color is `colors[i]`, coordinate is `coord[i]`.)

## Function
This subsection introduces the following functions that the tutorial use frequently:
- point cloud file I/O (tutlibs.io.Points)
- visualization of point clouds (tutlibs.visualization.JupyterVisualizer)

In this subsection, we use this package:

In [1]:
import numpy as np
import os
from tutlibs.visualization import JupyterVisualizer as jv
from tutlibs.io import Points


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### Point cloud file I/O (tutlibs.io.Points)

This tutorial mainly deal with vertex-only [PLY](http://paulbourke.net/dataformats/ply/) files as point cloud files. This subsection introduces the write and read functions of the PLY file.  
First, we introduce the read function, `Points.read`:


In [2]:
xyz, rgb, data = Points.read("../data/bunny_pc.ply")
print("XYZ shape:{}".format(xyz.shape))
print("RGB shape:{}".format(rgb.shape))
print("data type:{}, key:{}".format(type(data), data.keys()))


XYZ shape:(4093, 3)
RGB shape:(4093, 3)
data type:<class 'dict'>, key:dict_keys(['nx', 'ny', 'nz'])


`Points.read` reads data from a `ply` file. The return values of `Points.read` are `xyz`, `rgb`, and `data` :
1. `xyz`: a XYZ coordinate array. Shape of the array is (number of points, 3).
2. `rgb`: a RGB array. Shape of the array is (number of points, 3).
3. `data`: a file content. This return value is `dict` type, and may contain not only colors and coordinates, but also surface normals, etc.

Next, the write function, `Points.write` is as follows:

In [3]:
os.makedirs("outputs/", exist_ok=True)
Points.write(
    "outputs/basic_code_pc.ply", xyz, rgb, color_range=[0, 255], additional_data=None
)


`Points.write` write data to a `ply` file. The arguments of `Points.write` are as follows:
1. `filename`: file path to write a point cloud
2. `xyz`: a XYZ coordinate array. Shape of the array is (number of points, 3).
3. `rgb`: a RGB array. Shape of the array is (number of points, 3).
4. `color_range`: a RGB value range. The default range is [0, 255]. 
5. `additional_data`: additional data except RGB and XYZ coordinate data. This value is `dict` type or `None`, and default value is `None`.


### Visualization (tutlibs.visualization.JupyterVisualizer as jv)
This subsection introduce a visualization function:

In [3]:
obj_point = jv.point(xyz, rgb, color_range=[0, 255])
jv.display([obj_point])


Output()

We can get an object to display a point cloud from `jv.point`, and then `jv.display` displays point clouds from a list of objects. The arguments of `jv.point` are as follows:
1. `xyz`: a XYZ coordinate array. Shape of the array is (number of points, 3).
2. `colors`: a RGB array. Shape of the array is (number of points, 3).
3. `color_range`: a RGB value range. The default range is [0, 255]. 
4. `point_size`: point size for display

`jv` can display other 3D representation. For more `jv`, Please refer to [python/visualization.ipynb](python/visualization.ipynb).